# Volatility 25 (1h) Exploration

Early pass at understanding the synthetic index candle data coming from Deriv. These notebooks should remain lightweight and reproducible so they can be rerun whenever fresh data is ingested.


In [1]:
from __future__ import annotations

from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.width", 120)
pd.set_option("display.max_columns", None)
plt.style.use("ggplot")


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
DATA_PATH = Path("../data/raw/r_25_1h.csv")
assert DATA_PATH.exists(), f"Expected data file at {DATA_PATH}"  # ensure fetch step has run

df = pd.read_csv(DATA_PATH, parse_dates=["timestamp"])
df = df.sort_values("timestamp").reset_index(drop=True)

df.head()


In [ ]:
summary = df.describe(include="all", datetime_is_numeric=True)
summary


In [ ]:
df["close_return"] = df["close"].pct_change()
df["log_return"] = np.log(df["close"]).diff()
df["rolling_vol_24h"] = df["close_return"].rolling(window=24).std() * np.sqrt(24)

df[["timestamp", "close", "close_return", "rolling_vol_24h"]].tail(10)


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

axes[0].plot(df["timestamp"], df["close"], label="Close")
axes[0].plot(df["timestamp"], df["sma_20"], label="SMA 20")
axes[0].set_ylabel("Price")
axes[0].legend()
axes[0].set_title("Volatility 25 Close vs SMA20")

axes[1].plot(df["timestamp"], df["rolling_vol_24h"], color="tab:orange", label="Rolling Vol (24h)")
axes[1].set_ylabel("Annualised Vol")
axes[1].set_xlabel("Timestamp")
axes[1].legend()

fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
ax = df["log_return"].dropna().hist(bins=50, figsize=(10, 4))
ax.set_title("Distribution of Hourly Log Returns")
ax.set_xlabel("Log Return")
ax.set_ylabel("Frequency")
plt.show()


## Next Questions

- How stable are SMA crossovers on shorter horizons (e.g., 5m or 15m granularity)?
- What does performance look like for simple threshold strategies on log returns?
- Are there regime shifts we can detect via rolling volatility clustering?

Capture any deeper views of mine in `docs/journal.md` once experiments land in code/backtests.
